In [1]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Feature kelime listeleri
pozitif_kelimeler = [
    "harika", "mükemmel", "tavsiye", "rahat", "hızlı", "güzel", "ideal", "kaliteli",
    "süper", "beğendim", "memnun", "iyi", "şahane", "harikulade",  "keyifli", "ucuz"
]

negatif_kelimeler = [
    "kırık", "bozuk", "sert", "rahatsız", "berbat", "iade", "gıcırtı", "sıkıntı",
    "kötü", "patladı", "rezalet", "arıza", "dayanıksız", "paslı", 
]


# 1. JSON dosyasını oku
def load_reviews_from_json(path):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return [review.get("comment", "") for review in data["reviews"]]

# 2. Yukardaki kelimeleri kullanarak yorumların etiketlerini otomatik olarak belirleme
def auto_label(yorumlar):
    etiketler = []
    for yorum in yorumlar:
        y = yorum.lower()
        pos = any(poz in y for poz in pozitif_kelimeler)
        neg = any(neg in y for neg in negatif_kelimeler)
        if pos and not neg:
            etiketler.append(1)     # pozitif
        elif neg and not pos:
            etiketler.append(-1)    # negatif
        else:
            etiketler.append(0)     # nötr
    return etiketler

# 3. Model eğitimi
def egit_model(yorumlar, etiketler):
    vectorizer = TfidfVectorizer(max_features=3000, ngram_range=(1, 2))
    X = vectorizer.fit_transform(yorumlar)
    y = etiketler

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Eğitim sırasında class_weight kullan:
    model = LogisticRegression(multi_class='multinomial', max_iter=1000, class_weight='balanced')
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    print("\n--- Değerlendirme ---\n")
    print(classification_report(y_test, y_pred, zero_division=0))

    return model, vectorizer

# 4. Uygulama
yorumlar = load_reviews_from_json("response.json")
etiketler = auto_label(yorumlar)
model, vectorizer = egit_model(yorumlar, etiketler)



--- Değerlendirme ---

              precision    recall  f1-score   support

           0       0.85      0.85      0.85        13
           1       0.75      0.75      0.75         8

    accuracy                           0.81        21
   macro avg       0.80      0.80      0.80        21
weighted avg       0.81      0.81      0.81        21



c:\Users\ŞEVVAL\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [2]:
def tahmin_et(model, vectorizer, yeni_yorumlar):
    X_new = vectorizer.transform(yeni_yorumlar)
    tahminler = model.predict(X_new)
    return list(zip(yeni_yorumlar, tahminler))

# Örnek:
yeni = ["Bisiklet bozuk geldi", "Gerçekten çok kaliteli", "Koltuk biraz rahatsız ama idare eder", "Bisiklet yakıyor", 
"Bisiklet çok iyi", "Bisiklet çok iyi", "Berbat", "Süper", "Selesi çok sert"]
print(tahmin_et(model, vectorizer, yeni))


[('Bisiklet bozuk geldi', np.int64(1)), ('Gerçekten çok kaliteli', np.int64(1)), ('Koltuk biraz rahatsız ama idare eder', np.int64(0)), ('Bisiklet yakıyor', np.int64(1)), ('Bisiklet çok iyi', np.int64(1)), ('Bisiklet çok iyi', np.int64(1)), ('Berbat', np.int64(0)), ('Süper', np.int64(1)), ('Selesi çok sert', np.int64(0))]


In [3]:
from collections import Counter
print(Counter(etiketler))


Counter({0: 62, 1: 37, -1: 2})
